The line (x_train, y_train), (x_test, y_test) = mnist.load_data() is responsible for loading the MNIST dataset using the mnist.load_data() function from the Keras library.

The MNIST dataset is a widely used benchmark dataset in machine learning and computer vision. It consists of a large number of grayscale images of handwritten digits (0-9), along with their corresponding labels.

When mnist.load_data() is called, it returns four NumPy arrays:

x_train: This array contains the training images. It is a 3-dimensional array of shape (num_train_samples, height, width), where num_train_samples represents the number of training samples, and height and width represent the dimensions of each image (28x28 pixels).
y_train: This array contains the labels for the training images. It is a 1-dimensional array of shape (num_train_samples,), where num_train_samples represents the number of training samples.
x_test: This array contains the testing images. It has the same structure as x_train, with dimensions (num_test_samples, height, width).
y_test: This array contains the labels for the testing images. It has the same structure as y_train, with dimensions (num_test_samples,).
By unpacking the values returned by mnist.load_data() into the variables (x_train, y_train), (x_test, y_test), the code assigns the training and testing images and labels to the corresponding variables for further processing and model training

Three-layer network on MNIST

In [20]:
!pip  install tensorflow

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [21]:
!pip install keras

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [22]:
import sys, numpy as np
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

images, labels = (x_train[0:1000].reshape(1000,28*28) / 255, y_train[0:1000]) # image and labels are two vectors basically.

one_hot_labels = np.zeros((len(labels),10))
for i,l in enumerate(labels):
    one_hot_labels[i][l] = 1
labels = one_hot_labels

test_images = x_test.reshape(len(x_test),28*28) / 255
test_labels = np.zeros((len(y_test),10))

for i,l in enumerate(y_test):
    test_labels[i][l] = 1

np.random.seed(1)
relu = lambda x: (x > 0) * x
relu2deriv = lambda x: (x > 0)

alpha, iterations, hidden_size, pixels_per_image, num_labels = (0.005, 350, 40, 784, 10)

weights_0_1 = 0.2 * np.random.random((pixels_per_image,hidden_size)) - 0.1
weights_1_2 = 0.2 * np.random.random((hidden_size,num_labels)) - 0.1

for j in range(iterations):
    error, correct_cnt = (0.0,0)
    for i in range(len(images)):
        layer_0 = images[i:i+1]
        layer_1 = relu(np.dot(layer_0,weights_0_1))
        layer_2 = np.dot(layer_1,weigths_1_2)
        
        error+= np.sum((labels[i:i+1] - layer_2) ** 2)  # layer_2 --> output and labels --> goal_prediction
        
        correct_cnt += int(np.argmax(layer_2) == np.argmax(labels)) # if both equal then returns 1 else return 0.
        
        layer_2_delta = labels[i:i+1] - layer_2
        layer_1_delta = layer_2_delta.dot(weigths_1_2.T) * relu2deriv(layer_1)
        
        # now we can change our weights so that we can make the error == 0.
        
        layer_2_weight_delta = layer_1.T.dot(layer_2_delta)
        layer_1_weight_delta = layer_0.T.dot(layer_1_delta)
        
        weights_1_2 += alpha * layer_2_weight_delta
        weights_0_1 += alpha * layer_1_weight_delta
        
        sys.stdout.write("\r" + "I " + str(j) + "error: " + str(error/float(len(images)))[0:5] + "corrections " + str(correct_cnt/float(len(images))))
        

I 349error: 0.112corrections 0.092

droupout mask

In [26]:
import sys, numpy as np
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

images, labels = (x_train[0:1000].reshape(1000,28*28) / 255, y_train[0:1000])

one_hot_labels = np.zeros((len(labels),10))

for i,l in enumerate(labels):
    one_hot_labels[i][l] = 1
    
labels = one_hot_labels

test_images = x_test.reshape(len(x_test),28*28) / 255
test_labels = np.zeros((len(y_test),10))

for i,l in enumerate(y_test):
    test_labels[i][l] = 1

np.random.seed(1)

def relu(x):
    return (x > 0) * x
def relu2deriv(output):
    return output >= 0

batch_size = 100
alpha, iterations = (0.001, 300)
pixels_per_image, num_labels, hidden_size = (784, 10, 100)

weights_0_1 = 0.2 * np.random.random((pixels_per_image,hidden_size)) - 0.1
weights_1_2 = 0.2 * np.random.random((hidden_size,num_labels)) - 0.1

for j in range(iterations):
    error,correct_cnt = (0.0,0)
    for i in range(int(len(images) / batch_size)):
        batch_start, batch_end = ((i * batch_size),((i+1) * batch_size))
        
        layer_0 = images[batch_start:batch_end]
        layer_1 = relu(np.dot(layer_0,weights_0_1))
        droupout_mask = np.random.randint(2,size = layer_1.shape)
        layer_1 *= droupout_mask * 2
        layer_2 = np.dot(layer_1,weights_1_2)
        
        error += np.sum((labels[batch_start:batch_end] - layer_2) ** 2)
        
        for k in range(batch_size):
            correct_cnt += int(np.argmax(layer_2[k:k+1]) == np.argmax(labels[batch_start + k:batch_start + k+1]))
            
            layer_2_delta = (labels[batch_start:batch_end] - layer_2) / batch_size
            layer_1_delta = layer_2_delta.dot(weights_1_2.T) * relu2deriv(layer_1)
            
            layer_1_delta *= droupout_mask
            
            weights_1_2 += alpha * layer_1.T.dot(layer_2_delta)
            weights_0_1 += alpha * layer_0.T.dot(layer_1_delta)
            
    if(j % 10 == 0):
        test_error = 0.0
        test_correct_cnt = 0
        
        for i in range(len(test_images)):
            layer_0 = test_images[i:i+1]
            layer_1 = relu(np.dot(layer_0,weights_0_1))
            layer_2 = np.dot(layer_1,weights_1_2)
            
            test_error += np.sum((test_labels[i:i+1] - layer_2) ** 2)
            test_correct_cnt += int(np.argmax(layer_2) == np.argmax(test_labels[i:i+1]))
            
        sys.stdout.write("\n" + " I " + str(j) + " Test-Error " + str(test_error/ float(len(test_images)))[0:5] + " Test-Acc: " + str(test_correct_cnt/ float(len(test_images))) +" Train-Err: " + str(error/ float(len(images)))[0:5] +" Train-Acc: " + str(correct_cnt/ float(len(images))))
        


 I 0 Test-Error 0.815 Test-Acc: 0.3832 Train-Err: 1.284 Train-Acc: 0.165
 I 10 Test-Error 0.568 Test-Acc: 0.7173 Train-Err: 0.591 Train-Acc: 0.672
 I 20 Test-Error 0.510 Test-Acc: 0.7571 Train-Err: 0.532 Train-Acc: 0.729
 I 30 Test-Error 0.485 Test-Acc: 0.7793 Train-Err: 0.498 Train-Acc: 0.754
 I 40 Test-Error 0.468 Test-Acc: 0.7877 Train-Err: 0.489 Train-Acc: 0.749
 I 50 Test-Error 0.458 Test-Acc: 0.793 Train-Err: 0.468 Train-Acc: 0.775
 I 60 Test-Error 0.452 Test-Acc: 0.7995 Train-Err: 0.452 Train-Acc: 0.799
 I 70 Test-Error 0.446 Test-Acc: 0.803 Train-Err: 0.453 Train-Acc: 0.792
 I 80 Test-Error 0.451 Test-Acc: 0.7968 Train-Err: 0.457 Train-Acc: 0.786
 I 90 Test-Error 0.447 Test-Acc: 0.795 Train-Err: 0.454 Train-Acc: 0.799
 I 100 Test-Error 0.448 Test-Acc: 0.793 Train-Err: 0.447 Train-Acc: 0.796
 I 110 Test-Error 0.441 Test-Acc: 0.7943 Train-Err: 0.426 Train-Acc: 0.816
 I 120 Test-Error 0.442 Test-Acc: 0.7966 Train-Err: 0.431 Train-Acc: 0.813
 I 130 Test-Error 0.441 Test-Acc: 0.790